In [ ]:
from utils import preprocess_features, create_graph

import pandas as pd

In [ ]:
features = pd.read_csv('../data/large_twitch_features.csv')
edges = pd.read_csv('../data/large_twitch_edges.csv')

features = preprocess_features(features)
g, reverse_eids = create_graph(edges=edges, nodes=features)

In [ ]:
from utils import train, _initialise_training, _run_one_epoch

model, dataloader, optimizer = _initialise_training(
        g, reverse_eids, 'cuda',  
        n_hidden=64, learning_rate=1e-6, graph_sampling_size=[15, 5],
        negative_sample_size=3, weight_decay=1e-5, data_batch_size=512
    )

In [ ]:
len(dataloader)

In [ ]:
for epoch in range(5):
    print('epoch', epoch)
    model.train()
    model, optimizer, loss = _run_one_epoch(
        model=model,
        optimizer=optimizer,
        dataloader=dataloader,
        verbose = 2000,
        n_optimizer_steps=10000
    )
    print('loss', loss)

In [ ]:
import pickle
from dgl.data.utils import save_graphs

save_graphs('graph.bin', [g])

with open("model.pkl", "wb") as stream:
        pickle.dump(model, stream)

In [ ]:
import torch

with torch.no_grad():
    node_emb = model.inference(g, device='cuda').numpy()

# Predict Example

# Visualisations

In [ ]:
import pandas as pd

edges = pd.read_csv('../data/large_twitch_edges.csv')

In [ ]:
sample_edges = edges.sample(100)

import networkx as nx
G = nx.from_pandas_edgelist(
    sample_edges,
    source='numeric_id_1',
    target='numeric_id_2'
)

nx.draw_networkx(G)

In [ ]:
import networkx as nx
from pyviz.network import Network

G = nx.from_pandas_edgelist(
    sample_edges,
    source='numeric_id_1',
    target='numeric_id_2'
)
net = Network(notebook=True)
net.from_nx(G)
net.show('graph.html')